In [14]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import MetaPath2Vec
import torch_geometric.transforms as T
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import trange
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt

In [8]:
class network(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.lin = torch.nn.Linear(30, 20)
        self.out = torch.nn.Linear(20, 10)
    def forward(self, x):
        x = self.lin(x)
        x = F.relu(x)
        x = self.out(x)
        return x

net = network()
net_optimal = torch.optim.Adam(net.parameters(), lr=1e-3)

In [9]:
data1 = torch.normal(0, 1, size=(90000, 30))
data2 = torch.normal(3, 1, size=(90000, 30))
data = net(torch.cat([data1, data2], dim=0).requires_grad_())

In [10]:
# 调用k均值聚类方法
k = 2
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(data.detach().numpy())
centers = kmeans.cluster_centers_
labels = kmeans.labels_

In [15]:
ch_score = calinski_harabasz_score(data.detach().numpy(), labels)
db_score = davies_bouldin_score(data.detach().numpy(), labels)

In [13]:
ch_score

636111.9846337792

In [16]:
db_score

0.5026158376424148